# Road Following 
問題2では、問題1と同じ手順で行います！ 画像分類の代わりに、JetBotが道路（または実際には任意のパスまたはターゲットポイント）をたどることができるようにするために使用する、別の基本的な手法である回帰を学習します。

JetBotをパス上のさまざまな位置に配置します（中心からのオフセット、さまざまな角度など）
衝突回避から覚えておいてください、データのバリエーションが重要です！

ロボットからのライブカメラフィードを表示する
ゲームパッドコントローラーを使用して、ロボットに移動させたいターゲットの方向に対応する「緑の点」を画像上に配置します。
この緑色の点のX、Y値をロボットのカメラからの画像とともに保存します
次に、トレーニングノートブックで、ニューラルネットワークをトレーニングして、ラベルのX、Y値を予測します。ライブデモでは、予測されたX、Yの値を使用して、おおよそのステアリング値を計算します（画像のキャリブレーションが必要になるため、「正確に」角度ではありませんが、角度にほぼ比例するため、コントローラーは正常に動作します） ）。

それでは、この例のターゲットを正確にどこに配置するのでしょうか？ここに役立つと思われるガイドがあります

カメラからのライブビデオフィードを見てください
ロボットがたどるべき経路を想像してください（道路からの脱出などを回避するために必要な距離を概算してみてください）
ロボットが道路を「走る」ことなくターゲットにまっすぐ進むことができるように、ターゲットをこのパスに沿ってできるだけ遠くに配置します。
たとえば、非常にまっすぐな道路を走行している場合、水平線に配置できます。急旋回している場合は、境界の外に出ないようにロボットの近くに配置する必要があります。

ディープラーニングモデルが意図したとおりに機能すると仮定すると、これらのラベリングガイドラインでは次のことが保証されます。

ロボットは、ターゲットに向かって安全に直接移動できます（範囲外に出ることなく）。
目標は私たちの想像した道に沿って絶えず進歩
私たちが得るものは、私たちの望む軌道に沿って動く「スティック上のニンジン」です。ディープラーニングがニンジンを配置する場所を決定し、JetBotがそれに従います:)

### データ集めの様子

実際のデータ集めの様子を映像で確認してください
> コツは様々な角度から道の中央線にポインタを向けて撮影することです。

In [1]:
# from IPython.display import HTML
# HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


### ライブラリをインポートしています

それでは、「データ収集」のために必要なすべてのライブラリをインポートすることから始めましょう。 主にOpenCVを使用して、ラベル付きの画像を視覚化し保存します。 uuid、datetimeなどのライブラリは、イメージの命名に使用されます。

In [2]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
from uuid import uuid1
import json
import os
import glob
import datetime
import numpy as np
import cv2
import time

### Display Live Camera Feed

First, let's initialize and display our camera like we did in the teleoperation notebook. 

We use Camera Class from JetBot to enable CSI MIPI camera. Our neural network takes a 224x224 pixel image as input. We'll set our camera to that size to minimize the filesize of our dataset (we've tested that it works for this task). In some scenarios it may be better to collect data in a larger image size and downscale to the desired size later.

In [3]:
camera = Camera()

image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

# 緑色のポインタを移動させるためのバーを表示します
# バーは-1.0から1.0まで動かすことができます
x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    
    # バーはの値を取得しています
    x = x_slider.value
    y = y_slider.value
    
    # 座標が-1.0-1.0で表されているので、それを0-224に変更する
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    
    # カメラからの映像上に緑色のドットや直線を描画する
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(1)
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

# 作成したバーを表示する
display(widgets.HBox([image_widget, target_widget]), x_slider, y_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0, step=0.001)

FloatSlider(value=0.0, description='y', max=1.0, min=-1.0, step=0.001)

## 追加部分S2

それでは、保存用のボタンを作成しましょう。
サポートミッションwedgets.ipynbを参考にしよう

ボタンサイズは`width='128px', height='64px'`で設定し、ボタンのレイアウト設定は変数名を`save_button`にし`description='add blocked'`を`description='save'`に変更してください

In [4]:
# ボタンのサイズの設定

# ボタンのレイアウトの設定


それでは、作成したボタンを表示しましょう

In [5]:
display(widgets.HBox([save_button]))

NameError: name 'save_button' is not defined

それでは、データを収集します
画像を撮影する上での注意点を述べておきます

> ・ロードプレートの中央線に緑色のドットを置いて作成してください（1セル目の動画参照）

> ・様々な背景で画像を撮影したほうが精度は高くなります

> ・画像は40枚以上撮影してください

保存される画像は下記のようにxの座標、yの座標がファイル名になります
``xy_<x value>_<y value>_<uuid>.jpg``

トレーニングするときは、画像をロードし、ファイル名からx、y値を解析します

## 追加部分S2

それでは、撮影した画像を保存するディレクトリを作成していきましょう。
mission1でも使ったディレクトリの作成を参考にしてください。

ディレクトリ`dataset_xy`を作成し、変数名を`DATASET_DIR`にして下さい。

※今回は`dataset_xy`という名前のディレクトリがあればいいので、`free`,`blocked`ディレクトリは必要ありません。

※今回のディレクトリ名は`dataset/dataset_xy`ではなく、直接`dataset_xy`と指定してください。

In [ ]:
#DATASET_DIRという変数にdataset_xyを代入してください


try:
    #ディレクトリをプログラムを追加してください
    
except FileExistsError:
    print('Directories not created becasue they already exist')

それでは、撮影画像の合計枚数を表示しましょう

In [ ]:
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))
display(widgets.VBox([
    count_widget
]))

撮影して保存するメソッドの追加

In [ ]:
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

# 画像を保存している
def save_snapshot():
    uuid = xy_uuid(x_slider.value, y_slider.value)
    image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image_widget.value)
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

# ボタンと撮影するメソッドを紐づける
save_button.on_click(lambda x: save_snapshot())

それでは、撮影画像をzipファイルにしよう

In [ ]:
def timestr():
    return str(datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

!zip -r -q road_following_{DATASET_DIR}_{timestr()}.zip {DATASET_DIR}

You should see a file named road_following_<Date&Time>.zip in the Jupyter Lab file browser. You should download the zip file using the Jupyter Lab file browser by right clicking and selecting Download.